# Predicting which records match

In the previous tutorial, we built and estimated a linkage model.

In this tutorial, we will load the estimated model and use it to make predictions of which pairwise record comparisons match.


In [ ]:
# Uncomment and run this cell if you're running in Google Colab.
# !pip install git+https://github.com/moj-analytical-services/splink.git@splink4_examples_notebooks

In [1]:
from splink.linker import Linker
from splink.database_api import DuckDBAPI

from splink.datasets import splink_datasets

import pandas as pd

pd.options.display.max_columns = 1000

db_api = DuckDBAPI()
df = splink_datasets.fake_1000

## Load estimated model from previous tutorial


In [2]:
import json
import urllib

url = "https://raw.githubusercontent.com/moj-analytical-services/splink_demos/master/demo_settings/saved_model_from_demo.json"

with urllib.request.urlopen(url) as u:
    settings = json.loads(u.read().decode())


linker = Linker(df, settings, database_api=DuckDBAPI())

# Predicting match weights using the trained model

We use `linker.predict()` to run the model.

Under the hood this will:

- Generate all pairwise record comparisons that match at least one of the `blocking_rules_to_generate_predictions`

- Use the rules specified in the `Comparisons` to evaluate the similarity of the input data

- Use the estimated match weights, applying term frequency adjustments where requested to produce the final `match_weight` and `match_probability` scores

Optionally, a `threshold_match_probability` or `threshold_match_weight` can be provided, which will drop any row where the predicted score is below the threshold.


In [3]:
df_predictions = linker.predict(threshold_match_probability=0.2)
df_predictions.as_pandas_dataframe(limit=5)


 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'email':
    m values not fully trained


,match_weight,match_probability,unique_id_l,unique_id_r,first_name_l,first_name_r,gamma_first_name,bf_first_name,surname_l,surname_r,gamma_surname,bf_surname,dob_l,dob_r,gamma_dob,bf_dob,city_l,city_r,gamma_city,tf_city_l,tf_city_r,bf_city,bf_tf_adj_city,email_l,email_r,gamma_email,bf_email,match_key
0,-1.221199,0.300174,453,454,Davies,Davies,4,84.391544,Reuben,Reuben,4,88.44144,1988-01-20,1987-11-21,2,3.332963,Swasea,Swansea,0,0.002460,0.015990,0.464956,1.000000,rd@lewis.com,idlewrs.cocm,0,0.011093,0
1,-1.193795,0.304179,601,602,Toby,Toby,4,84.391544,Hall,None,-1,1.00000,2003-04-23,2013-03-21,1,0.588068,London,London,1,0.212792,0.212792,10.167006,0.259162,toby.h@mendez.com,None,-1,1.000000,0
2,-1.193795,0.304179,600,602,Toby,Toby,4,84.391544,None,None,-1,1.00000,2003-04-23,2013-03-21,1,0.588068,London,London,1,0.212792,0.212792,10.167006,0.259162,toby.d@menhez.com,None,-1,1.000000,0
3,-1.193795,0.304179,599,602,Toby,Toby,4,84.391544,Haall,None,-1,1.00000,2003-04-23,2013-03-21,1,0.588068,London,London,1,0.212792,0.212792,10.167006,0.259162,None,None,-1,1.000000,0
4,-1.193795,0.304179,324,326,Kai,Kai,4,84.391544,None,Turner,-1,1.00000,2018-12-31,2009-11-03,1,0.588068,London,London,1,0.212792,0.212792,10.167006,0.259162,k.t50eherand@z.ncom,None,-1,1.000000,0


## Clustering

The result of `linker.predict()` is a list of pairwise record comparisons and their associated scores. For instance, if we have input records A, B, C and D, it could be represented conceptually as:

```
A -> B with score 0.9
B -> C with score 0.95
C -> D with score 0.1
D -> E with score 0.99
```

Often, an alternative representation of this result is more useful, where each row is an input record, and where records link, they are assigned to the same cluster.

With a score threshold of 0.5, the above data could be represented conceptually as:

```
ID, Cluster ID
A,  1
B,  1
C,  1
D,  2
E,  2
```

The algorithm that converts between the pairwise results and the clusters is called connected components, and it is included in Splink. You can use it as follows:


In [4]:
clusters = linker.cluster_pairwise_predictions_at_threshold(
    df_predictions, threshold_match_probability=0.5
)
clusters.as_pandas_dataframe(limit=10)

Completed iteration 1, root rows count 11
Completed iteration 2, root rows count 1
Completed iteration 3, root rows count 0


,cluster_id,unique_id,first_name,surname,dob,city,email,cluster,__splink_salt,tf_city
0,0,0,Robert,Alan,1971-06-24,None,robert255@smith.net,0,0.273376,NaN
1,0,1,Robert,Allen,1971-05-24,None,roberta25@smith.net,0,0.349102,NaN
2,0,2,Rob,Allen,1971-06-24,London,roberta25@smith.net,0,0.916980,0.212792
3,0,3,Robert,Alen,1971-06-24,Lonon,None,0,0.211071,0.007380
4,4,4,Grace,None,1997-04-26,Hull,grace.kelly52@jones.com,1,0.033932,0.001230
5,4,5,Grace,Kelly,1991-04-26,None,grace.kelly52@jones.com,1,0.461907,NaN
6,6,6,Logan,pMurphy,1973-08-01,None,None,2,0.561858,NaN
7,7,7,None,None,2015-03-03,Portsmouth,evied56@harris-bailey.net,3,0.483785,0.017220
8,8,8,None,Dean,2015-03-03,None,None,3,0.460459,NaN
9,8,9,Evie,Dean,2015-03-03,Pootsmruth,evihd56@earris-bailey.net,3,0.897049,0.001230


In [5]:
sql = f"""
select *
from {df_predictions.physical_name}
limit 2
"""
linker.query_sql(sql)

,match_weight,match_probability,unique_id_l,unique_id_r,first_name_l,first_name_r,gamma_first_name,bf_first_name,surname_l,surname_r,gamma_surname,bf_surname,dob_l,dob_r,gamma_dob,bf_dob,city_l,city_r,gamma_city,tf_city_l,tf_city_r,bf_city,bf_tf_adj_city,email_l,email_r,gamma_email,bf_email,match_key
0,-1.221199,0.300174,453,454,Davies,Davies,4,84.391544,Reuben,Reuben,4,88.44144,1988-01-20,1987-11-21,2,3.332963,Swasea,Swansea,0,0.002460,0.015990,0.464956,1.000000,rd@lewis.com,idlewrs.cocm,0,0.011093,0
1,-1.193795,0.304179,601,602,Toby,Toby,4,84.391544,Hall,None,-1,1.00000,2003-04-23,2013-03-21,1,0.588068,London,London,1,0.212792,0.212792,10.167006,0.259162,toby.h@mendez.com,None,-1,1.000000,0


!!! note "Further Reading"
:material-tools: For more on the prediction tools in Splink, please refer to the [Prediction API documentation](../../linkerpred.md).


## Next steps

Now we have made predictions with a model, we can move on to visualising it to understand how it is working.
